In [1]:
import import_ipynb
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
import os
import time
import datetime
import logging
import argparse
import numpy as np
import torch
import torch.nn.functional as F
from torch import distributed as dist
import torchvision
from torchvision import datasets, models, transforms
from configs.default_img_visualization import get_img_config
from models.img_resnet import ResNet50
from data import build_dataloader
from models.classifier import Classifier, NormalizedClassifier
from PIL import Image
import gc
from tools.utils import set_seed

In [3]:
import sys

sys.argv = [
    'demo_single_image.ipynb',
    '--cfg', 'configs/res50_cels_cal.yaml',
    '--dataset', 'prcc', 
    '--img_path', 'C:/Users/USER/OneDrive/Documents/My Nural Net/Person Re-Identification/dataset/prcc/rgb/test/B/030/cropped_rgb442.jpg',
    '--weights', 'C:/Users/USER/OneDrive/Documents/My Nural Net/Person Re-Identification/outputs/prcc/eval/best_model.pth.tar', 
    '--gpu', '0',
]

In [ ]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

In [5]:
def parse_option():
    parser = argparse.ArgumentParser(
        description='Train clothes-changing re-id model with clothes-based adversarial loss')
    parser.add_argument('--cfg', type=str, required=True, metavar="FILE", help='path to config file')
    # Datasets
    parser.add_argument('--root', type=str, help="your root path to data directory")
    parser.add_argument('--dataset', type=str, default='ltcc', help="ltcc, prcc, vcclothes, last, deepchange")
    # Miscs
    parser.add_argument('--img_path', type=str, help='path to the image')
    parser.add_argument('--weights', type=str, help='path to the weights')
    parser.add_argument('--gpu', type=str, default='0', help='gpu id')

    args, unparsed = parser.parse_known_args()
    config = get_img_config(args)
    return config, args

In [ ]:
@torch.no_grad()
def extract_img_feature(model, img):
    flip_img = torch.flip(img, [3])
    img, flip_img = img.to(device), flip_img.to(device)
    _, batch_features = model(img)
    _, batch_features_flip = model(flip_img)
    batch_features += batch_features_flip
    batch_features = F.normalize(batch_features, p=2, dim=1)
    features = batch_features.cpu()

    return features

config, args = parse_option()

dict = torch.load(args.weights)
model = ResNet50(config)

model.load_state_dict(dict['model_state_dict'])
model = model.to(device)
model.eval()

data_transforms = transforms.Compose([
        transforms.Resize((config.DATA.HEIGHT, config.DATA.WIDTH)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

image = Image.open(args.img_path)
image_tensor = data_transforms(image)
input_batch = image_tensor.unsqueeze(0)  

feature = extract_img_feature(model, input_batch)

print("Input Image:", args.img_path, " Output Feautre:", feature)
print(feature.shape) 

In [ ]:
import cv2
import matplotlib.pyplot as plt

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.feature_maps = None

        # Register hooks
        self.target_layer.register_forward_hook(self.save_feature_maps)
        self.target_layer.register_backward_hook(self.save_gradients)

    def save_feature_maps(self, module, input, output):
        self.feature_maps = output

    def save_gradients(self, module, grad_input, grad_output):
        self.gradients = grad_output[0]

    def generate_cam(self, input_tensor):
        # Forward pass
        base_f, f = self.model(input_tensor)
        # Compute the loss with respect to a dummy target
        loss = f.norm()  
        self.model.zero_grad()
        loss.backward()

        # Get gradients and feature maps
        gradients = self.gradients.cpu().data.numpy()[0]
        feature_maps = self.feature_maps.cpu().data.numpy()[0]

        # Compute the weights
        weights = np.mean(gradients, axis=(1, 2))
        cam = np.sum(weights[:, np.newaxis, np.newaxis] * feature_maps, axis=0)

        # Apply ReLU to the CAM
        cam = np.maximum(cam, 0)
        cam = cam - np.min(cam)
        cam = cam / np.max(cam)
        cam = cv2.resize(cam, (input_tensor.size(2), input_tensor.size(3)))
        return cam

# Set up Grad-CAM
target_layer = model.layer4  
grad_cam = GradCAM(model, target_layer)

input_batch = input_batch.to(device)

# Generate the heatmap
heatmap = grad_cam.generate_cam(input_batch)
heatmap = cv2.resize(heatmap, (image.size[0], image.size[1]))

# Convert the heatmap to an image
#heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_RAINBOW) 
heatmap = np.float32(heatmap) / 255
overlayed_img = heatmap + np.array(image.resize((heatmap.shape[1], heatmap.shape[0]))) / 255
overlayed_img = overlayed_img / np.max(overlayed_img)

# Plot the original image and the heatmap
plt.figure(figsize=(5, 5))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.axis('off')
plt.title("Original Image")
plt.subplot(1, 2, 2)
plt.imshow(overlayed_img)
plt.axis('off')
plt.title("Heatmap Overlay")
plt.show()